In [1]:
from pymongo import MongoClient
from elasticsearch import Elasticsearch, helpers
import requests
from IPython.display import JSON
import numpy as np
from fuzzywuzzy import fuzz
from joblib import Parallel, delayed
import pickle
from IPython.display import clear_output, display
from unidecode import unidecode
import re

In [2]:
def parse_doi(doi:str):
    return re.sub(r'https*\:\/\/[\w\.]+\/','',doi.lower())

def paser_string(string:str):
    data = str(string).lower()
    data = unidecode(data)
    data = re.sub(r'<[^>]+>','',data)
    return data

In [3]:
scopus = list(MongoClient()["scopus_colombia"]["stage"].find({'DOI': {"$ne": np.nan}},
                                                             {"_id":0,'Title':1,'Year':1, 
                                                              'Source title':1,'Authors':1,"DOI":1,
                                                             "Volume":1, "Issue":1, "Page start":1, "Page end":1}))
openalex = list(MongoClient()["openalexco"]["works"].find({'doi': {"$ne": None}},{"_id":0,"doi":1}))

In [4]:
JSON(openalex[0])

<IPython.core.display.JSON object>

In [5]:
len(openalex)

203863

In [6]:
len(scopus)

143369

In [8]:
%%time
#positives
def get_p(i):    
    for j in openalex:
        if parse_doi(i["DOI"]) ==  parse_doi(j['doi']):
            return i
p = Parallel(n_jobs=72,backend='multiprocessing',verbose=5)(delayed(get_p)(rec) for rec in scopus)


[Parallel(n_jobs=72)]: Using backend MultiprocessingBackend with 72 concurrent workers.
[Parallel(n_jobs=72)]: Done  18 tasks      | elapsed:    0.1s
[Parallel(n_jobs=72)]: Done 176 tasks      | elapsed:    1.2s
[Parallel(n_jobs=72)]: Done 542 tasks      | elapsed:    3.1s
[Parallel(n_jobs=72)]: Done 1296 tasks      | elapsed:    6.1s
[Parallel(n_jobs=72)]: Done 1729 tasks      | elapsed:    7.4s
[Parallel(n_jobs=72)]: Done 2302 tasks      | elapsed:    9.3s
[Parallel(n_jobs=72)]: Done 2801 tasks      | elapsed:   11.3s
[Parallel(n_jobs=72)]: Done 3219 tasks      | elapsed:   12.2s
[Parallel(n_jobs=72)]: Done 3876 tasks      | elapsed:   13.9s
[Parallel(n_jobs=72)]: Done 4914 tasks      | elapsed:   17.7s
[Parallel(n_jobs=72)]: Done 5790 tasks      | elapsed:   19.4s
[Parallel(n_jobs=72)]: Done 6768 tasks      | elapsed:   22.3s
[Parallel(n_jobs=72)]: Done 7812 tasks      | elapsed:   25.6s
[Parallel(n_jobs=72)]: Done 8776 tasks      | elapsed:   29.8s
[Parallel(n_jobs=72)]: Done 9378 

CPU times: user 3min 48s, sys: 1min 30s, total: 5min 18s
Wall time: 18min 40s


[Parallel(n_jobs=72)]: Done 143369 out of 143369 | elapsed: 18.7min finished


In [9]:
p.count(None)

41596

In [10]:
p=[i for i in p if i is not None ]

In [11]:
p[0]

{'Authors': 'Jaramillo-Ayerbe F., Berrío-Muñoz J.',
 'Title': 'Ivermectin for crusted Norwegian scabies induced by use of topical steroids',
 'Year': 1998,
 'Source title': 'Archives of Dermatology',
 'Volume': '134',
 'Issue': '2',
 'Page start': '143',
 'Page end': '145',
 'DOI': '10.1001/archderm.134.2.143'}

In [12]:
#los que no tienen un doi en oa, osea los que no están en p_dois

p_dois = [i["DOI"] for i in p]

In [13]:
len(p_dois)

101773

In [14]:
%%time
#negatives
n=[]
counter=0
n_dois = []
for i in scopus:
    if i["DOI"] not in p_dois:
        n.append(i)
        counter+=1
        if counter %10 == 0:
            clear_output(wait=True)
            print(counter)

41590
CPU times: user 2min 36s, sys: 2.58 s, total: 2min 39s
Wall time: 2min 36s


In [15]:
len(p)

101773

In [16]:
len(n)

41596

In [17]:
len(scopus)

143369

In [18]:
len(p)+len(n)

143369

In [19]:
with open('dataset_full.pkl', 'wb') as f:
    pickle.dump({"p":p,"n":n}, f)